In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geopy.distance import great_circle
import random

# Load datasets
vts_df = pd.read_csv(r"C:\Users\rekha\OneDrive\Desktop\Toll\Sankari-BPCL.csv")
report_df = pd.read_csv(r"C:\Users\rekha\OneDrive\Desktop\Toll\KL_NHAI_Toll_Plazas_Without_Toll_Amount_1.csv")
toll_rates_df = pd.read_csv(r"C:\Users\rekha\OneDrive\Desktop\Toll\rateofthetoll.csv")

# Function to calculate distance between two points
def calculate_distance(coord1, coord2):
    return great_circle(coord1, coord2).kilometers

# Function to normalize names
def normalize_name(name):
    return name.strip().lower()

# Function to generate a random color
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Normalize the names in the toll rates dataframe
toll_rates_df['location'] = toll_rates_df['location'].apply(normalize_name)

# Dictionary to track the last toll plaza crossed by each vehicle
last_crossed_toll = {}

# DataFrame to store results for all trips
all_trip_data = []

# Group data by trip_id to process each trip separately
grouped_vts_df = vts_df.groupby('trip_id')

# Process each trip
for trip_id, trip_data in grouped_vts_df:
    crossed_plaza_names = []
    single_journey_costs = []
    roundtrip_costs = []
    vts_coordinates_list = []
    unique_trip_color = get_random_color()
    roundtrip_detected = False

    # Create a base Folium map
    mapping_the_variable = folium.Map(location=[vts_df['Latitude'].mean(), vts_df['Longitude'].mean()], zoom_start=10)

    # Create a MarkerCluster object
    marker_cluster = MarkerCluster().add_to(mapping_the_variable)

    for _, vts_row in trip_data.iterrows():
        vts_coordinates = (vts_row['Latitude'], vts_row['Longitude'])
        vehicle_id = vts_row['vehicle_number']
        company_name = vts_row.get('Terminal', 'Unknown')  # Use 'Unknown' if 'Terminal' is missing

        # Add the truck's location to the map
        folium.CircleMarker(location=vts_coordinates, radius=3, color=unique_trip_color).add_to(mapping_the_variable)
        vts_coordinates_list.append(vts_coordinates)

        # Check if the truck has crossed any toll plaza
        for idx, row in report_df.iterrows():
            toll_coordinates = (row['Latitude'], row['Longitude'])
            toll_plaza_name = normalize_name(row['Toll Plaza Name'])  # Normalize the name for comparison
            toll_location = row['Toll Plaza Location']
            state = row['State']
            sr_no = row['Sr No.']

            distance = calculate_distance(vts_coordinates, toll_coordinates)

            # Assuming a crossing is detected if distance is less than 3.0 km
            if distance < 3.0:  # Adjust this threshold based on your actual data and requirements
                if last_crossed_toll.get(vehicle_id) != toll_plaza_name:
                    crossed_plaza_names.append(toll_plaza_name)
                    last_crossed_toll[vehicle_id] = toll_plaza_name

                    # Check if the toll plaza has already been crossed (indicating a roundtrip)
                    if toll_plaza_name in crossed_plaza_names[:-1]:
                        roundtrip_detected = True

                    # Find the toll cost from the toll rates dataframe using the toll plaza name as the location
                    toll_cost_row = toll_rates_df[toll_rates_df['location'] == toll_plaza_name]
                    if not toll_cost_row.empty:
                        if roundtrip_detected:
                            roundtrip_costs.append(toll_cost_row.iloc[0]['Single Journey'])
                        else:
                            single_journey_costs.append(toll_cost_row.iloc[0]['Single Journey'])

    # Add all toll plaza markers to the map
    for idx, row in report_df.iterrows():
        toll_coordinates = (row['Latitude'], row['Longitude'])
        toll_plaza_name = row['Toll Plaza Name']
        toll_location = row['Toll Plaza Location']
        state = row['State']
        sr_no = row['Sr No.']

        # Check for missing values and replace them with 'Unknown'
        toll_plaza_name = toll_plaza_name if pd.notna(toll_plaza_name) else 'Unknown'
        toll_location = toll_location if pd.notna(toll_location) else 'Unknown'
        state = state if pd.notna(state) else 'Unknown'
        sr_no = sr_no if pd.notna(sr_no) else 'Unknown'

        # Create a tooltip with toll details
        tooltip = f"Sr No.: {sr_no}<br>Name: {toll_plaza_name}<br>Location: {toll_location}<br>State: {state}"

        # Add a marker with a red-colored icon and tooltip to the MarkerCluster
        folium.Marker(
            location=toll_coordinates,
            popup=toll_plaza_name,
            tooltip=tooltip,
            icon=folium.Icon(color='red', icon_size=(35, 35))  # Adjust the icon size here
        ).add_to(marker_cluster)

    # Add the route line to the map for the current trip with a popup showing the trip ID
    folium.PolyLine(vts_coordinates_list, color='blue', weight=5, popup=f'Trip ID: {trip_id}').add_to(mapping_the_variable)

    # Save the map to an HTML file named with the trip ID
    html_file_path = rf"C:\Users\rekha\OneDrive\Desktop\Toll\maps\Trip_{trip_id}.html"
    mapping_the_variable.save(html_file_path)

    # Calculate total trip costs
    total_single_journey_cost = sum(single_journey_costs)
    total_roundtrip_cost = sum(roundtrip_costs)
    total_trip_cost = total_single_journey_cost + total_roundtrip_cost  # Total cost including both single journey and roundtrip costs

    # Calculate roundtrip return journey costs
    return_journey_costs = []

    if roundtrip_detected:
        # Reverse the crossed plaza names to calculate return journey costs
        crossed_plaza_names.reverse()

        for toll_plaza_name in crossed_plaza_names:
            toll_cost_row = toll_rates_df[toll_rates_df['location'] == toll_plaza_name]
            if not toll_cost_row.empty:
                return_journey_costs.append(toll_cost_row.iloc[0]['Single Journey'])

    total_return_journey_cost = sum(return_journey_costs)

    # Append trip data to the list
    trip_data_dict = {
        'Trip ID': trip_id,
        'Vehicle Number': vehicle_id,
        'Company Name': company_name,
        'Crossed Toll Plazas Names': crossed_plaza_names,
        'Single Journey Costs': single_journey_costs,
        'Roundtrip Costs': roundtrip_costs,
        'Total Trip Cost': total_trip_cost,
        'Roundtrip Detected': roundtrip_detected
    }
    all_trip_data.append(trip_data_dict)

# Convert the list of dictionaries to a DataFrame
all_trip_df = pd.DataFrame(all_trip_data)

# Split the data into two DataFrames based on whether any tolls were crossed
crossed_tolls_df = all_trip_df[all_trip_df['Crossed Toll Plazas Names'].str.len() > 0]
not_crossed_tolls_df = all_trip_df[all_trip_df['Crossed Toll Plazas Names'].str.len() == 0]

# Specify the full path where you want to save the CSV files
crossed_tolls_csv_path = r"C:\Users\rekha\OneDrive\Desktop\Toll\Crossed_Tolls_Data1.csv"
not_crossed_tolls_csv_path = r"C:\Users\rekha\OneDrive\Desktop\Toll\Not_Crossed_Tolls_Data1.csv"

# Save the DataFrames to CSV files
crossed_tolls_df.to_csv(crossed_tolls_csv_path, index=False)
not_crossed_tolls_df.to_csv(not_crossed_tolls_csv_path, index=False)

print(f"Crossed tolls data saved to {crossed_tolls_csv_path}")
print(f"Not crossed tolls data saved to {not_crossed_tolls_csv_path}")


Crossed tolls data saved to C:\Users\rekha\OneDrive\Desktop\Toll\Crossed_Tolls_Data1.csv
Not crossed tolls data saved to C:\Users\rekha\OneDrive\Desktop\Toll\Not_Crossed_Tolls_Data1.csv
